In [2]:
#!/usr/bin/python3
import argparse
from random import uniform, randint

def generate_problem(
    variables=10, 
    constraint_probability=0.25, 
    R=100,
    min_intervals=1, 
    max_intervals=5,
    scaling_factor=1,
    ):
    
    T_all = []
    
    for j in range(variables):
        T_all.append({
            'i': 0,
            'j': j,
            'intervals': [(-R, R)],
        })
        
    for i in range(1, variables):
        for j in range(i+1, variables):
            if uniform(0, 1) < constraint_probability:
                num_intervals = randint(min_intervals, max_intervals)
                points = [randint(-R, R) for _ in range(2 * num_intervals)]
                points.sort()
                
                intervals = list(zip(points[::2], points[1::2]))
                
                # apply scaling factor:
                intervals = [ (i, int(i + (j-i)*scaling_factor)) for i, j in intervals]
                
                T_all.append({
                    'i': i,
                    'j': j,
                    'intervals': intervals,
                })

    return T_all

In [ ]:
def genetic(num_iterations):
    
    # generate random gene pool
    
    for it in num_iterations:
        # rank gene pool
        # discard lowest 80%
        # mutate and mix best
        
        # if some satisfies, return it
        
    
    # return (No, best ranked gene)